In [ ]:
import os
from getpass import getpass
from google import genai

# pass your API key here
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY") or getpass(
    "Enter Google API Key: "
)
# initialize our client
client = genai.Client()

In [2]:
from IPython.display import Markdown

model_id = "gemini-2.0-flash-exp"

response = client.models.generate_content(
    model=model_id, contents='What is Gemini?'
)
Markdown(response.text)

Gemini refers to a few different things, but most commonly it refers to **Google's large language model.** Here's a breakdown:

**1. Gemini (The Google AI Model):**

*   **What it is:** Gemini is a multimodal AI model developed by Google AI. It's designed to be Google's most capable and general AI model, surpassing previous models like LaMDA and PaLM.
*   **Capabilities:** It can understand and generate text, images, audio, video, and code. This "multimodal" nature means it can process and understand information from various sources simultaneously. It's designed to be highly adaptable and perform well on a wide range of tasks.
*   **Versions:** There are different versions of Gemini optimized for various use cases:
    *   **Gemini Ultra:** The most powerful version, designed for highly complex tasks.
    *   **Gemini Pro:** A more balanced version, suitable for a wide range of applications, including powering Google's Bard (now Gemini) chatbot.
    *   **Gemini Nano:** A lightweight version designed to run on devices like smartphones and other edge devices.
*   **Use Cases:** Gemini is being integrated into various Google products and services, including:
    *   **Search:** Improving search results and providing more comprehensive answers.
    *   **Gmail & Workspace:** Helping with email composition, summarizing documents, and generating ideas.
    *   **Bard/Gemini Chatbot:** Providing more natural and helpful conversational AI.
    *   **Coding:** Assisting developers with code generation, debugging, and understanding code.
    *   **Creative Content:** Helping with writing, image generation, and other creative tasks.

**2. Gemini (The Zodiac Sign):**

*   In astrology, Gemini is the third sign of the zodiac, associated with people born between approximately May 21 and June 20. It's an air sign often associated with communication, curiosity, and adaptability.

**3. Project Gemini (NASA):**

*   In the 1960s, Project Gemini was a NASA human spaceflight program that served as a bridge between Project Mercury (the first US human spaceflight program) and Project Apollo (the program that landed humans on the Moon). Gemini developed techniques for rendezvous, docking, and spacewalking that were crucial for the success of Apollo.

**In summary, when someone mentions "Gemini" today, they are most likely referring to the Google AI model.**  To avoid confusion, it's helpful to consider the context in which the word is used.


In [4]:
from google.genai.types import Tool, GoogleSearch

search_tool = Tool(google_search=GoogleSearch())

In [ ]:
from google.genai.types import GenerateContentConfig
model_id = "gemini-2.0-flash-exp"

config = GenerateContentConfig(
    system_instruction=(
        "You are a helpful assistant that provides up to date information "
        "to help the user in their research."
    ),
    tools=[search_tool],
    response_modalities=["TEXT"],
    temperature=0.0,  # likely default
    candidate_count=1,  # likely default

)
config
response = client.models.generate_content(
    model=model_id,
    contents="Tell me the latest news in AI",
    config=config
)
Markdown(response.text)

GenerateContentConfig(http_options=None, system_instruction='You are a helpful assistant that provides up to date information to help the user in their research.', temperature=0.0, top_p=None, top_k=None, candidate_count=1, max_output_tokens=None, stop_sequences=None, response_logprobs=None, logprobs=None, presence_penalty=None, frequency_penalty=None, seed=None, response_mime_type=None, response_schema=None, routing_config=None, model_selection_config=None, safety_settings=None, tools=[Tool(retrieval=None, google_search=GoogleSearch(), google_search_retrieval=None, enterprise_web_search=None, google_maps=None, code_execution=None, function_declarations=None)], tool_config=None, labels=None, cached_content=None, response_modalities=['TEXT'], media_resolution=None, speech_config=None, audio_timestamp=None, automatic_function_calling=None, thinking_config=None)

In [ ]:
response = client.models.generate_content(
    model=model_id,
    contents="Tell me the latest news in AI",
    config=config
)
Markdown(response.text)

In [9]:
response = client.models.generate_content(
    model="gemini-2.0-flash-exp",
    contents="Tell me the latest news in AI",
    config={
        "tools": [{"google_search": {}}],
        "temperature": 0,
        },
    )
Markdown(response.text)

KeyboardInterrupt: 

In [ ]:
import os


In [ ]:
import os


from langchain_core.tools import Tool
from langchain_google_community import GoogleSearchAPIWrapper

search = GoogleSearchAPIWrapper()

tool = Tool(
    name="google_search",
    description="Search Google for recent results.",
    func=search.run,
)

In [11]:
tool.run("Obama's first name?")

KeyboardInterrupt: 

In [2]:
from langchain_tavily import TavilySearch
from langgraph.prebuilt import create_react_agent
from langchain_deepseek import ChatDeepSeek
from dotenv import load_dotenv

load_dotenv()
llm = ChatDeepSeek(
    model="deepseek-chat",
    temperature=1,
    max_tokens=1024,
    timeout=None,
    max_retries=2,
    )


# Initialize Tavily Search Tool
tavily_search_tool = TavilySearch(
    max_results=5,
    topic="general",
)

agent = create_react_agent(llm, [tavily_search_tool])

user_input = "What nation hosted the Euro 2024? Include only wikipedia sources."

for step in agent.stream(
    {"messages": user_input},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What nation hosted the Euro 2024? Include only wikipedia sources.
================================== Ai Message ==================================
Tool Calls:
  tavily_search (call_0_edc53b20-0873-4a26-a204-8448a393fc9c)
 Call ID: call_0_edc53b20-0873-4a26-a204-8448a393fc9c
  Args:
    query: Euro 2024 host nation
    include_domains: ['wikipedia.org']
================================= Tool Message =================================
Name: tavily_search

{"query": "Euro 2024 host nation", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "UEFA Euro 2024 - Wikipedia", "url": "https://en.wikipedia.org/wiki/UEFA_Euro_2024", "content": "It was the third time that European Championship matches were played on German territory, and the second time in reunified Germany, as West Germany hosted the 1988 tournament, and four matches of the multi-national Euro 2020 were played in Munich. 

In [17]:
load_dotenv()
llm = ChatDeepSeek(
    model="deepseek-chat",
    temperature=1,
    max_tokens=1024,
    timeout=None,
    max_retries=2,
    )
# Create a ReAct agent with the web searcher system prompt and Tavily search tool
formatted_prompt = web_searcher_instructions.format(
    current_date=get_current_date(),
    research_topic='price of 5090',
)

# Create the agent with the formatted prompt as system message
agent = create_react_agent(
    llm,
    [tavily_search_tool],
    prompt=formatted_prompt
)

# Create a runnable chain from the agent
# search_chain = agent.get_chain()

# agent.invoke({"messages": [HumanMessage(content="What is the price of 5090?")]}).pretty_print()

user_input = "What is the price of 5090?"

for step in agent.stream(
    {"messages": user_input},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()


================================ Human Message =================================

What is the price of 5090?
================================== Ai Message ==================================
Tool Calls:
  tavily_search (call_0_db1ab490-3839-48ef-bd9b-fa5fe82a4e53)
 Call ID: call_0_db1ab490-3839-48ef-bd9b-fa5fe82a4e53
  Args:
    query: price of 5090
    search_depth: advanced
    time_range: week
================================= Tool Message =================================
Name: tavily_search

{"query": "price of 5090", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://www.tomshardware.com/pc-components/gpus/cheapest-5090-ever-lucky-pc-builder-finds-rtx-5090-for-just-usd1-679-at-walmart-saving-usd1-520-card-has-worked-flawlessly-ever-since", "title": "Cheapest 5090 ever? Lucky PC builder finds RTX 5090 for 'just ...", "content": "Redditor Sufficient\\_Crazy7758 took to the platform to hail their purchase of the \"Cheapest 5090 ever?\", after scoo

In [3]:
import os

from agent.tools_and_schemas import SearchQueryList, Reflection
from dotenv import load_dotenv
from langchain_core.messages import AIMessage
from langgraph.types import Send
from langgraph.graph import StateGraph
from langgraph.graph import START, END
from langchain_core.runnables import RunnableConfig
from google.genai import Client

from agent.state import (
    OverallState,
    QueryGenerationState,
    ReflectionState,
    WebSearchState,
)
from agent.configuration import Configuration
from agent.prompts import (
    get_current_date,
    query_writer_instructions,
    web_searcher_instructions,
    reflection_instructions,
    answer_instructions,
)
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_deepseek import ChatDeepSeek

from agent.utils import (
    get_citations,
    get_research_topic,
    insert_citation_markers,
    resolve_urls,
)


In [4]:

def generate_query(state: OverallState, config: RunnableConfig) -> QueryGenerationState:
    """LangGraph node that generates a search queries based on the User's question.

    Uses Gemini 2.0 Flash to create an optimized search query for web research based on
    the User's question.

    Args:
        state: Current graph state containing the User's question
        config: Configuration for the runnable, including LLM provider settings

    Returns:
        Dictionary with state update, including search_query key containing the generated query
    """
    configurable = Configuration.from_runnable_config(config)

    # check for custom initial search query count
    if state.get("initial_search_query_count") is None:
        state["initial_search_query_count"] = configurable.number_of_initial_queries

    # init Gemini 2.0 Flash
    # llm = ChatGoogleGenerativeAI(
    #     model=configurable.query_generator_model,
    #     temperature=1.0,
    #     max_retries=2,
    #     api_key=os.getenv("GEMINI_API_KEY"),
    # )
    llm = ChatDeepSeek(
    model="deepseek-chat",
    temperature=1,
    max_tokens=1024,
    timeout=None,
    max_retries=2,
    )
    structured_llm = llm.with_structured_output(SearchQueryList)

    # Format the prompt
    current_date = get_current_date()
    formatted_prompt = query_writer_instructions.format(
        current_date=current_date,
        research_topic=get_research_topic(state["messages"]),
        number_queries=state["initial_search_query_count"],
    )
    # Generate the search queries
    result = structured_llm.invoke(formatted_prompt)
    return {"query_list": result.query}

In [ ]:
def continue_to_web_research(state: QueryGenerationState):
    """LangGraph node that sends the search queries to the web research node.

    This is used to spawn n number of web research nodes, one for each search query.
    """
    return [
        Send("web_research", {"search_query": search_query, "id": int(idx)})
        for idx, search_query in enumerate(state["query_list"])
    ]


def web_research(state: WebSearchState, config: RunnableConfig) -> OverallState:
    """LangGraph node that performs web research using the native Google Search API tool.

    Executes a web search using the native Google Search API tool in combination with Gemini 2.0 Flash.

    Args:
        state: Current graph state containing the search query and research loop count
        config: Configura"tion for the runnable, including search API settings

    Returns:
        Dictionary with state update, including sources_gathered, research_loop_count, and web_research_results
    """
    # Configure
    configurable = Configuration.from_runnable_config(config)
    formatted_prompt = web_searcher_instructions.format(
        current_date=get_current_date(),
        research_topic=state["search_query"],
    )
    llm = ChatDeepSeek(
    model="deepseek-chat",
    temperature=1,
    max_tokens=1024,
    timeout=None,
    max_retries=2,
    )

    # Uses the google genai client as the langchain client doesn't return grounding metadata
    response = genai_client.models.generate_content(
        model=configurable.query_generator_model,
        contents=formatted_prompt,
        config={
            "tools": [{"google_search": {}}],
            "temperature": 0,
        },
    )

    
    # resolve the urls to short urls for saving tokens and time
    resolved_urls = resolve_urls(
        response.candidates[0].grounding_metadata.grounding_chunks, state["id"]
    )
    # Gets the citations and adds them to the generated text
    citations = get_citations(response, resolved_urls)
    modified_text = insert_citation_markers(response.text, citations)
    sources_gathered = [item for citation in citations for item in citation["segments"]]

    return {
        "sources_gathered": sources_gathered,
        "search_query": [state["search_query"]],
        "web_research_result": [modified_text],
    }

In [ ]:
builder = StateGraph(OverallState, config_schema=Configuration)

# Define the nodes we will cycle between
builder.add_node("generate_query", generate_query)
builder.add_node("web_research", web_research)
builder.add_edge(START, "generate_query")
builder.add_conditional_edges(
    "generate_query", continue_to_web_research, ["web_research"]
)

builder.add_edge("finalize_answer", END)

graph = builder.compile(name="pro-search-agent")


In [ ]:
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.messages import AIMessage, HumanMessage

class LLMGeneratedSearchQueries(BaseModel):
    """Schema for the LLM to generate a list of search queries."""
    search_queries: list[str] = Field(description="A list of search queries to find relevant information based on the research topic.")

llm = ChatDeepSeek(
    model="deepseek-chat",
        temperature=0.0,
        max_tokens=1024,
        timeout=None,
        max_retries=2,
    )

messages = [HumanMessage(content=formatted_prompt)]

query_generation_response = llm.invoke(
    messages,
    tools=[LLMGeneratedSearchQueries]
)

generated_queries = []
if query_generation_response.tool_calls:
    for tool_call in query_generation_response.tool_calls:
        if tool_call['name'] == LLMGeneratedSearchQueries.__name__:
            generated_queries.extend(tool_call['args'].get('search_queries', []))

TypeError: argument of type 'ModelMetaclass' is not iterable